In [1]:
from allennlp.models.archival import load_archive
from chemdataextractor.data import find_data
import json
import pprint

In [2]:
overrides = {"model.text_field_embedder.token_embedders.bert.pretrained_model": find_data("models/scibert_cased_weights-1.0.tar.gz")}
cde_bert_archive = load_archive(find_data('models/bert_finetuned_crf_model-1.0a'), overrides=json.dumps(overrides))

In [3]:
cde_bert_model = cde_bert_archive.model
pprint.pprint(cde_bert_model)

_BertCrfTagger(
  (text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_bert): PretrainedBertEmbedder(
      (bert_model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(31116, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (output): BertSelfOutput(
          

In [4]:
cde_bert_model_state_dict = cde_bert_model._modules['text_field_embedder']._modules['token_embedder_bert']._modules['bert_model'].state_dict()

In [5]:
from transformers import AutoModel, AutoTokenizer
scibert_tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_cased")
scibert_model = AutoModel.from_pretrained("allenai/scibert_scivocab_cased")
scibert_model.load_state_dict(cde_bert_model_state_dict, strict=False)

Some weights of the model checkpoint at allenai/scibert_scivocab_cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_IncompatibleKeys(missing_keys=['embeddings.position_ids'], unexpected_keys=[])

In [12]:
cde_bert_model.label_namespace

'labels'

In [15]:
from allennlp.modules.conditional_random_field import allowed_transitions

In [18]:
labels = cde_bert_model.vocab.get_index_to_token_vocabulary('labels')
print(labels)
allowed_transitions('BIO', labels)

{0: 'O', 1: 'I-CEM', 2: 'B-CEM'}


[(0, 0),
 (0, 2),
 (0, 4),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 4),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 4),
 (3, 0),
 (3, 2)]